In [1]:
# Imports
import pandas as pd
import numpy as np
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression, RidgeCV, LassoCV, ElasticNetCV
from sklearn.metrics import mean_squared_error, make_scorer
from scipy.stats import skew
from IPython.display import display
import matplotlib.pyplot as plt
import seaborn as sns


In [2]:
# Get data
train = pd.read_csv("train.csv")
test = pd.read_csv('test.csv')

# Drop columns
train.drop(["Id","Street","PoolQC","Utilities","MiscFeature","MiscVal", "Alley"], axis = 1, inplace = True)
test.drop(["Id","Street","PoolQC","Utilities","MiscFeature","MiscVal", "Alley"],axis = 1, inplace = True)

## removing outliers
train= train[train.YearBuilt> 1895]
train = train[train.GrLivArea < 4500]

#### IMPUTING  NAs #####

train['MasVnrType'].fillna(value = 'None', inplace = True)
train['MasVnrArea'].replace(np.nan, 0, inplace = True)
train['Fence'].fillna('None', inplace = True)
train['FireplaceQu'].fillna('None', inplace = True)

## garage columns
train['GarageType'].fillna('None', inplace = True)
train['GarageCond'].fillna('None', inplace = True)
train['GarageFinish'].fillna('None', inplace = True)
train['GarageQual'].fillna('None', inplace = True)
train['GarageYrBlt'].fillna(train['LotFrontage'].median(), inplace = True)


### basement columns
train['BsmtCond'].fillna('None', inplace = True)
train['BsmtQual'].fillna('None', inplace = True)
train['BsmtExposure'].fillna('None', inplace = True)
train['BsmtFinType2'].fillna('None', inplace = True)
train['BsmtFinType1'].fillna('None', inplace = True)
## use median for lot frontage
train['LotFrontage'].fillna((train['LotFrontage'].median()), inplace=True)
# use median for masvnrarea
abc1 = train['MasVnrArea']
abc1 = abc1.replace(0, np.nan)
abc1 = abc1.dropna(how='all', axis=0)
train['MasVnrArea'] = train['MasVnrArea'].replace(0,abc1.median())

# use median for garage area
abc2 = train['GarageArea']
abc2 = abc2.replace(0, np.nan)
abc2 = abc2.dropna(how='all', axis=0)
train['GarageArea'] = train['GarageArea'].replace(0,abc2.median())

# use median for total bsmt sf
abc3 = train['TotalBsmtSF']
abc3 = abc3.replace(0, np.nan)
abc3 = abc3.dropna(how='all', axis=0)
train['TotalBsmtSF'] = train['TotalBsmtSF'].replace(0,abc2.median())

## use mode for the 1 missing electrical value 
train['Electrical'].fillna((train['Electrical'].value_counts().index[0]), inplace = True)


#### ordinal columns ####

## Basement cols
# Basement Condition
mapping4 = {"None" : 0, "Po" : 1, "Fa" : 2, "TA" : 3, "Gd" : 4, "Ex" : 5}
train['BsmtCond'] = train.replace({'BsmtCond': mapping4}, inplace = True)
# Basement Exposure
mapping5 = {"No" : 0,'None':0, "Mn" : 1, "Av": 2, "Gd" : 3}
train['BsmtExposure'] = train.replace({'BsmtExposure': mapping5}, inplace = True)
# Basement Qual
mapping6 = {"None" : 0, "Po" : 1, "Fa" : 2, "TA": 3, "Gd" : 4, "Ex" : 5}
train['BsmtQual'] = train.replace({'BsmtQual':mapping6}, inplace = True)
# Basement Fin type 1
mapping7 = {"None": 0, "Unf" : 1, "LwQ": 2, "Rec" : 3, "BLQ" : 4, "ALQ" : 5, "GLQ" : 6}
train['BsmtFinType1'] = train.replace({'BsmtFinType1':mapping7}, inplace = True)
# Basement Fin type 2
mapping8 = {"None": 0, "Unf" : 1, "LwQ": 2, "Rec" : 3, "BLQ" : 4, "ALQ" : 5, "GLQ" : 6}
train['BsmtFinType2']= train.replace({'BsmtFinType2':mapping8}, inplace = True)

# Heating QC (quality)
mapping9 = {"Po" : 1, "Fa" : 2, "TA" : 3, "Gd" : 4, "Ex" : 5}
train['HeatingQC'] = train.replace({'HeatingQC':mapping9},inplace = True)

## Garage cols
# garage cond
mapping10 = {"None" : 0, "Po" : 1, "Fa" : 2, "TA" : 3, "Gd" : 4, "Ex" : 5}
train['GarageCond'] = train.replace({'GarageCond': mapping10}, inplace = True)
# garage qual
mapping11 = {"None" : 0, "Po" : 1, "Fa" : 2, "TA" : 3, "Gd" : 4, "Ex" : 5}
train['GarageQual'] = train.replace({'GarageQual': mapping11}, inplace = True)

### home functionality
mapping12 = {"Sal" : 1, "Sev" : 2, "Maj2" : 3, "Maj1" : 4, "Mod": 5, 
                                       "Min2" : 6, "Min1" : 7, "Typ" : 8}
train['Functional'] = train.replace({'Functional': mapping12}, inplace = True)

## exterior
# exterior condition
mapping13 = {"Po" : 1, "Fa" : 2, "TA": 3, "Gd": 4, "Ex" : 5}
train['ExterCond']= train.replace({'ExterCond': mapping13}, inplace = True)
# exterior quality
mapping14 = {"Po" : 1, "Fa" : 2, "TA": 3, "Gd": 4, "Ex" : 5}
train['ExterQual'] = train.replace({'ExterQual':mapping14}, inplace = True)

# kitchen quality
mapping15 = {"Po" : 1, "Fa" : 2, "TA" : 3, "Gd" : 4, "Ex" : 5}
train['KitchenQual']=train.replace({"KitchenQual": mapping15}, inplace = True)

# fireplace quality
mapping16 = {"None" : 0, "Po" : 1, "Fa" : 2, "TA" : 3, "Gd" : 4, "Ex" : 5}
train['FireplaceQu']= train.replace({'FireplaceQu': mapping16}, inplace = True)

# pool area
train['PoolArea'] = train['PoolArea'].apply(lambda row: 2 if (row > 640) else 1 if (row > 480) else 0)

# 3 ssn porch using median to categorize
train['3SsnPorch'] = train['3SsnPorch'].apply(lambda row: 0 if (row == 0) else 2 if (row > 200) else 1)

# low qual fin sf using median to categorize
train['LowQualFinSF'] = train['LowQualFinSF'].apply(lambda row: 0 if (row == 0) else 2 if (row > 377.5) else 1)

# screen porch using median to categorize
train['ScreenPorch'] = train['ScreenPorch'].apply(lambda row: 0 if (row == 0) else 2 if (row > 180) else 1)

# bsmt fin sf 2 using median to categorize
train['BsmtFinSF2'] = train['BsmtFinSF2'].apply(lambda row: 0 if (row == 0) else 2 if (row > 355) else 1)

# enclosed porch using median to categorize
train['EnclosedPorch'] = train['EnclosedPorch'].apply(lambda row: 0 if (row == 0) else 2 if (row > 144.5) else 1)

# 2nd floor SF using median to categorize
train['2ndFlrSF'] = train['2ndFlrSF'].apply(lambda row: 0 if (row == 0) else 2 if (row > 776) else 1)

# wood deck SF using median to categorize
train['WoodDeckSF'] = train['WoodDeckSF'].apply(lambda row: 0 if (row == 0) else 2 if (row > 171) else 1)

# open porch SF using median to categorize
train['OpenPorchSF'] = train['OpenPorchSF'].apply(lambda row: 0 if (row == 0) else 2 if (row > 63) else 1)

# bsmt fin sf 1 using median to categorize
train['BsmtFinSF1'] = train['BsmtFinSF1'].apply(lambda row: 0 if (row == 0) else 2 if (row > 604) else 1)

# bsmt unf sf using median to categorize
train['BsmtUnfSF'] = train['BsmtUnfSF'].apply(lambda row: 0 if (row == 0) else 2 if (row > 536) else 1)

#### NEW FEATURES ####

#total SF
train['TotalSF'] = train['BsmtFinSF1'] + train['BsmtFinSF2'] + train['1stFlrSF'] + train['2ndFlrSF']

# total baths
train['TotalBaths'] = train['FullBath']+train['BsmtFullBath']+(.5*train['HalfBath'])+(.5*train['BsmtHalfBath'])

# median price by year
medianbyyear = train.groupby('YrSold')['SalePrice'].agg('median').to_dict()
train['MedianPriceByYear'] = train.YrSold.apply(lambda x : medianbyyear[x])

# assign values to neighborhood
neighbOrder = train.groupby('Neighborhood',as_index = False)['SalePrice'].mean()
neighbID = pd.qcut(neighbOrder['SalePrice'], 8, labels=False)
newNeighb = pd.concat([neighbOrder['Neighborhood'],neighbID], axis =1)
neighb_dict = dict(zip(newNeighb.Neighborhood, newNeighb.SalePrice))
train['Neighborhood'] = train.Neighborhood.apply(lambda x: neighb_dict[x])

# was it remodeled
# was it sold same year it was built
train['Remodeled'] = (train['YearBuilt']==train['YearRemodAdd']).astype(int)
train['NewHouse'] = (train['YearBuilt']==train['YrSold']).astype(int)


### drop redundant columns
train = train.drop(columns = ['BsmtFinSF1','BsmtFinSF2','1stFlrSF','2ndFlrSF','FullBath','HalfBath',
                             'BsmtFullBath','BsmtHalfBath'])



# separate categorical and numerical feats
categorical_features = train.select_dtypes(include = ["object"]).columns
numerical_features = train.select_dtypes(exclude = ["object"]).columns
train_num = train[numerical_features]
train_cat = train[categorical_features]
### handling skewed numerical columns

# log skewness with an absolute value > 0.5
skewness = train_num.apply(lambda x: skew(x))
skewness = skewness[abs(skewness) > 0.5]
print(str(skewness.shape[0]) + " skewed numerical features to log transform")
skewed_features = skewness.index
train_num[skewed_features] = np.log1p(train_num[skewed_features])

# Create dummy features for categorical values via one-hot encoding
train_cat = pd.get_dummies(train_cat)
# Join categorical and numerical features
train = pd.concat([train_num, train_cat], axis = 1)


23 skewed numerical features to log transform


C:\Users\jdsip\anaconda3\lib\site-packages\pandas\core\frame.py:3391: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[k1] = value[k2]


In [3]:
train.shape

(1444, 185)

In [8]:
# train.to_csv('WithLogsAndNewFeats.csv')